<div style="border: 2px solid #255c8a; padding: 10px; background-color: #121212; border-radius: 5px; line-height: 1; width: 97%">
  <strong> ℹ️ Descrição:</strong> 
  Precisamos descobrir se existem dados que desvirtuam-se do padrão estabelecido nas partidas (podendo ser tanto entradas incorretas quanto entradas que podemm ocorrer, mas possuem uma chance baixíssima de acontecer), de forma a não prejudicar o modelo final.
</div>

> OBS: acho que teremos que refazer esse outliers, mas passando um por um. Talvez passar apenas pelas que vamos inputar? Para não serem tantas.

# Inicialização

**Bibliotecas**:

In [346]:
import os
import sys
import pandas as pd
from scipy.stats import zscore

**Funções Auxiliares:**

In [347]:
sys.path.append(os.path.abspath("../src"))
from AnaliseExploratoria_functions import histogram_and_stats

**Leitura do Arquivo:**

In [348]:
current_path = os.path.dirname(os.getcwd())
csv_path = (current_path + '/Data/campeonatos_futebol_sem_duplicados.csv')

df = pd.read_csv(csv_path)

# Identificando outliers

Podemos identificar outliers de diversas formas.  
Aqui, usaremos três técnicas

1. Percentis máximos e mínimos;
2. Intervalo interquartil;
3. Z-score

### Gráficos para auxiliar (falta organizar)

Para auxiliar na identificação das variáveis, e checar valores delas, vamos adicionar alguns gráficos para facilitar a visualização

> OBS: acho que esses são os gráficos que a IDE do joão faz bonitinho, certo? Temos que determinar se realmente vamos entregar assim, se vamos reescrever as funções para fazer gráficos parecidos só que em outras IDEs, ou se vamos descartar. Também queria melhorar o comentário acima, mas não consigo pensar em nada agora.

In [349]:
print(df.columns) # todas as colunas do dataframe original

Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'Cartões amarelos 1', 'Cartões amarelos 2',
       'Cartões vermelhos 1', 'Cartões vermelhos 2', 'Cruzamentos 1',
       'Cruzamentos 2', 'Laterais 1', 'Laterais 2', 'Chutes bloqueados 1',
       'Chutes bloqueados 2', 'Contra-ataques 1', 'Contra-ataques 2', 'Gols 1',
       'Gols 2', 'Tiro de meta 1', 'Tiro de meta 2', 'Tratamentos 1',
       'Tratamentos 2', 'Substituições 1', 'Substituições 2', 'Tiros-livres 1',
       'Tiros-livres 2', 'Defesas difíceis 1', 'Defesas difíceis 2',
       'Posse 1(%)', 'Posse 2(%)', 'Position 1', 'Position 2'],
      dtype='object')


In [350]:
# Vamos selecionar apenas as colunas com valores numéricos e exibir suas estatísticas

In [351]:
for col in df.select_dtypes(include="number").columns:
    print(f'\033[1m{col}\033[0m')
    histogram_and_stats(df,col,plot=False)

Chutes a gol 1
NaN's: 1512 --|-- Ocorrências: 26192 --|-- Média: 4.77 --|-- Desv. padrão: 2.80 --|-- Mínimo: 0.00 --|-- Máximo: 90.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 3.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 6.00 --|-- 95º percentil: 10.00
--------------
Chutes a gol 2
NaN's: 1512 --|-- Ocorrências: 26192 --|-- Média: 3.85 --|-- Desv. padrão: 2.44 --|-- Mínimo: 0.00 --|-- Máximo: 80.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 2.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 5.00 --|-- 95º percentil: 8.00
--------------
Impedimentos 1
NaN's: 2772 --|-- Ocorrências: 24932 --|-- Média: 2.14 --|-- Desv. padrão: 1.75 --|-- Mínimo: 0.00 --|-- Máximo: 23.00 --|-- 5º percentil: 0.00 --|-- 25º percentil: 1.00 --|-- 50º percentil (mediana): 2.00 --|-- 75º percentil: 3.00 --|-- 95º percentil: 5.00
--------------
Impedimentos 2
NaN's: 2772 --|-- Ocorrências: 24932 --|-- Média: 1.97 --|-- Desv. padrão: 1.68 --|-- Mínimo: 0.00 --|-- Máximo: 15.00 -

### Método 1: Percentis

Pelos gráficos da seção anterior, conseguimos perceber a existência de valores máximos bastante discrepantes (outliers). Então, como uma das estratégias para tratamento dos dados, realizamos o corte de todos os dados que estão acima do percentil 99.9 (representam o 0.01% maiores dados). 

In [352]:
# Fazendo um loop pelos dados acima do percentil 99.9
df_numerico = df.select_dtypes(include='number')

for col in df_numerico.columns:
    p999 = df[col].quantile(0.999)
    outliers_col = df[df[col] > p999]

    print(f"\n📊 Coluna: {col}")
    print(f"🔎 Quantidade de outliers (> P999 = {p999:.2f}): {len(outliers_col)}")

    # Mostra os outliers da coluna (apenas as colunas relevantes)
    if not outliers_col.empty:
        display(outliers_col[[col]])
    else:
        print("✅ Nenhum outlier encontrado acima do P999.")



📊 Coluna: Chutes a gol 1
🔎 Quantidade de outliers (> P999 = 16.00): 23


,Chutes a gol 1
137,19.0
220,90.0
698,18.0
1037,19.0
1061,20.0
2101,21.0
3313,17.0
5753,17.0
6388,18.0
13492,19.0



📊 Coluna: Chutes a gol 2
🔎 Quantidade de outliers (> P999 = 14.00): 22


,Chutes a gol 2
352,16.0
605,15.0
664,22.0
718,15.0
730,16.0
1997,17.0
2101,15.0
2410,80.0
2413,30.0
9751,16.0



📊 Coluna: Impedimentos 1
🔎 Quantidade de outliers (> P999 = 10.00): 19


...

,Defesas difíceis 2
5753,14.0
6388,17.0
6715,13.0
24433,14.0



📊 Coluna: Posse 1(%)
🔎 Quantidade de outliers (> P999 = 83.00): 22


,Posse 1(%)
68,85.0
819,100.0
841,100.0
842,100.0
846,100.0
859,100.0
871,100.0
2457,100.0
6869,100.0
7001,100.0



📊 Coluna: Posse 2(%)
🔎 Quantidade de outliers (> P999 = 80.00): 25


,Posse 2(%)
75,81.0
113,83.0
571,99.0
868,100.0
2305,100.0
2841,100.0
3148,100.0
3183,100.0
3997,100.0
6837,100.0


### Método 2: Distância Interquartil

O segundo método que utilizamos para encontrar outliers foi o da distância interquartil (IQR). Ele se baseia, resumidamente, em estabelercermos o primeiro quartil (q1), que separa os 25% menores valores do conjunto, e o terceiro quartil (q3), que separa os 75% menores valores. Então, estabelecemos a disância entre os dois quartis, IQR = q3 - q1, e calculamos os limites inferiores e superiores, estabelecendo o que seria considerável "aceitável" para aquele conjunto específico de dados. 

Entretanto, após a realização de alguns testes, percebemos que ao fixarmos q1 e q3, estávamos obtendo, como outliers, valores que não seriam impossíveis ou muito difíceis de se obter em uma partida real de futebol. Assim, decidimos aumentar a distância, na tentativa de encontrar, de fato, dados discrepantes. 

Após mais alguns testes, encontramos os percentis de 99 e 0.01 como ideais para aplicar esse método, considerando-se o conjunto de dados presente. 

> OBS: Tenho dúvida se esse método realmente pode ser utilizado assim. Pelo que me lembrava, a distância interquartílica era sempre em q3 e q1. 

In [353]:

# Pelo Métdo do Intervalo Quartil 

# O IQR (Intervalo Interquartil) é uma medida estatística que mostra a dispersão central dos dados, ignorando os valores extremos. Ele é muito usado para detectar outliers.
# - Q1 (1º quartil): valor que separa os 25% menores valores
# - Q3 (3º quartil): valor que separa os 75% menores valores
# - O IQR seria IQR = Q3-Q1

# Definimos o limite para outliers:
# Limite  = Q3 + 1.5 × IQR
# Qualquer valor fora desse intervalo é considerado outlier.

# Portanto, aplicando o métdo

df_numerico = df.select_dtypes(include='number')
for col in df_numerico.columns:
    q001 = df[col].quantile(0.01)
    q99 = df[col].quantile(0.99)
    iqr = q99 - q001

    limite_inferior = q001 - 1.5 * iqr
    limite_superior = q99 + 1.5 * iqr

    outliers = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]

    print(f"\n📊 Coluna: {col}")
    print(f"🔎 Outliers encontrados: {len(outliers)}")

    if not outliers.empty:
        print(outliers[[col]].head())  # Mostra só os primeiros outliers
    else:
        print("✅ Nenhum outlier detectado.")



📊 Coluna: Chutes a gol 1
🔎 Outliers encontrados: 1
     Chutes a gol 1
220            90.0

📊 Coluna: Chutes a gol 2
🔎 Outliers encontrados: 2
      Chutes a gol 2
2410            80.0
2413            30.0

📊 Coluna: Impedimentos 1
🔎 Outliers encontrados: 1
       Impedimentos 1
17320            23.0

📊 Coluna: Impedimentos 2
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Escanteios 1
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Escanteios 2
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Chutes fora 1
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Chutes fora 2
🔎 Outliers encontrados: 1
       Chutes fora 2
21347          748.0

📊 Coluna: Faltas 1
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Faltas 2
🔎 Outliers encontrados: 3
      Faltas 2
218       90.0
2410     180.0
2414     120.0

📊 Coluna: Cartões amarelos 1
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Cartões amarel

### Método 3: Z-score

O terceiro e último método utilizado é o dos z-scores. Através de uma padronização, ele mede o quão distante (em número de desvios padrão) um valor está da média da sua distribuição. 

Sua fórmula é:

$$z = (x−μ)/σ$$

Onde z é o z-score, x é o valor sendo analisado, $\mu$ é sua esperança, e $\sigma$ é o desvio padrão.  Além disso, vale citar que os NaNs foram ignorados no cálculo do Z-score

In [354]:
df_numerico = df.select_dtypes(include='number')
for col in df_numerico.columns:
    z = zscore(df[col], nan_policy='omit')  # calcula z-score, ignorando NaNs
    z_series = pd.Series(z, index=df.index)
    outliers = df[z_series.abs() > 4]

    print(f'\nOutliers na coluna "{col}":')
    print(outliers[[col]])


Outliers na coluna "Chutes a gol 1":
       Chutes a gol 1
137              19.0
220              90.0
301              16.0
698              18.0
1037             19.0
1061             20.0
1097             16.0
2071             16.0
2101             21.0
3313             17.0
5753             17.0
5775             16.0
6388             18.0
6715             16.0
6743             16.0
7019             16.0
7666             16.0
8959             16.0
9346             16.0
11249            16.0
13492            19.0
13514            19.0
16084            17.0
16927            17.0
17265            19.0
17267            18.0
17268            23.0
17335            21.0
18512            17.0
21128            18.0
21622            16.0
24800            16.0
25108            18.0
25315            16.0
25517            16.0
25716            16.0
25986            16.0
26149            18.0
26954            17.0
27354            17.0
27399            16.0
27595            16.0

Outliers na col

# Retirando outliers detectados

Após o uso destes três métodos, identificamos as seguintes linhas como outliers:

> Perguntar como exatamente foi feito isso (no olho? uma função?)

In [355]:
outliers = (218, 220, 423, 571, 819, 841, 842, 846, 859, 868, 871, 1257,
            1490, 1497, 2305, 2410, 2414, 2457, 2841, 3148, 3183, 3997,
            5150, 6837, 6869, 6911, 7001, 7003, 7004, 7005, 7771, 10651,
            10718, 10819, 10923, 11084, 11406, 11568, 12005, 12185, 13750,
            13812, 14283, 15532, 19611, 19616, 19622, 19652, 19655, 19827,
            21357, 22641, 22669, 23085, 23133, 24787, 24788, 24838, 24855)

df.iloc[list(outliers)]


,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Substituições 1,Substituições 2,Tiros-livres 1,Tiros-livres 2,Defesas difíceis 1,Defesas difíceis 2,Posse 1(%),Posse 2(%),Position 1,Position 2
218,5.0,5.0,0.0,0.0,7.0,2.0,17.0,4.0,10.0,90.0,...,3.0,3.0,NaN,NaN,NaN,NaN,67.0,33.0,4-2-3-1,4-5-1
220,90.0,3.0,0.0,0.0,5.0,5.0,10.0,5.0,13.0,14.0,...,3.0,3.0,NaN,NaN,NaN,NaN,52.0,48.0,4-2-3-1,4-2-3-1
423,0.0,3.0,0.0,2.0,2.0,7.0,4.0,9.0,46.0,13.0,...,3.0,3.0,NaN,NaN,NaN,NaN,51.0,49.0,3-5-2,4-2-3-1
571,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,99.0,4-4-2,4-4-2
819,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.0,3-5-2,4-1-4-1
841,4.0,0.0,NaN,NaN,0.0,7.0,0.0,9.0,0.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.0,4-4-2,5-3-2
842,NaN,NaN,3.0,0.0,NaN,NaN,5.0,0.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.0,4-4-1-1,4-4-2
846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.0,3-5-2,4-2-3-1
859,0.0,0.0,0.0,0.0,NaN,NaN,3.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.0,3-5-2,4-3-3
868,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100.0,4-4-2,4-2-3-1


Então, vamos eliminá-los (dar drop()) do data frame

In [356]:
df_sem_outliers = df.drop(df.index[list(outliers)])

# Retirando outros dados

Também podemos analisar e buscar dados que sejam impossíveis de ocorrer, mas que não foram detectados pelos outliers. 

> OBS: Isso é um problema, pois era para termos detectado. Após analisar um pouco, acredito que estejamos sendo muito restritos no que consideramos um outlier, especialmente no método da IQR. 

Um desses casos é o que temos 0% de posse de bola:

In [357]:
df_sem_outliers = df_sem_outliers[(df_sem_outliers['Posse 1(%)'] != 0) & (df_sem_outliers['Posse 2(%)'] != 0)]

In [358]:
# Os outliers de chutes fora 2 permaneceram quando fiz [4]. Acho que o intervalo para detectar esses outliers ficou muito pequeno. Depois temos que ver isso.

col = "Chutes fora 2"
q1 = df[col].quantile(0.25)
q3 = df[col].quantile(0.75)

IQR = q3 - q1

limite_inferior = q1 - 1.5*IQR
limite_superior = q3 + 1.5*IQR


outliers = df_sem_outliers[(df_sem_outliers[col] < limite_inferior) | (df_sem_outliers[col] > limite_superior)]
df_sem_outliers = df_sem_outliers.drop(outliers.index)

# Salvando novo csv

Então, vamos salvá-los em um novo arquivo:

In [359]:
df_sem_outliers.to_csv(current_path + '/Data/campeonatos_futebol_sem_outliers.csv', index=False)

# Novos gráficos

Finalmente, podemos analisar novamente os gráficos montados, desta vez sem os outliers:

In [360]:
for col in df.select_dtypes(include="number").columns:
    print(f'\033[1m{col}\033[0m')
    histogram_and_stats(df,col,plot=False)

Chutes a gol 1
NaN's: 1512 --|-- Ocorrências: 26192 --|-- Média: 4.77 --|-- Desv. padrão: 2.80 --|-- Mínimo: 0.00 --|-- Máximo: 90.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 3.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 6.00 --|-- 95º percentil: 10.00
--------------
Chutes a gol 2
NaN's: 1512 --|-- Ocorrências: 26192 --|-- Média: 3.85 --|-- Desv. padrão: 2.44 --|-- Mínimo: 0.00 --|-- Máximo: 80.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 2.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 5.00 --|-- 95º percentil: 8.00
--------------
Impedimentos 1
NaN's: 2772 --|-- Ocorrências: 24932 --|-- Média: 2.14 --|-- Desv. padrão: 1.75 --|-- Mínimo: 0.00 --|-- Máximo: 23.00 --|-- 5º percentil: 0.00 --|-- 25º percentil: 1.00 --|-- 50º percentil (mediana): 2.00 --|-- 75º percentil: 3.00 --|-- 95º percentil: 5.00
--------------
Impedimentos 2
NaN's: 2772 --|-- Ocorrências: 24932 --|-- Média: 1.97 --|-- Desv. padrão: 1.68 --|-- Mínimo: 0.00 --|-- Máximo: 15.00 -